In [167]:
# Delay Line Interface

In [ ]:
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
sys.path.append("C:/Users/alber/Downloads")  # PATH to hardware_ops.py

In [2]:
from hardware_ops import HardwareOps

In [3]:
from hardware_ops import HardwareOps

config_filename =  r'C:\Users\alber\Documents\hardware_project\hardware_config.json' # use this if you ned to store serial port numbers
ops = HardwareOps(config_filename) # initialized inside the __init__ functions

In [ ]:
# Zero Piezo Controllers to 40V
piezo_list = np.array(ops.piezo_serials_list)
x_piezo_list = piezo_list[[0,2]]

ops.piezos.set_to_voltage_zero_value(x_piezo_list, 40)

In [ ]:
delta = 10 # Voltage change

def compute_sensitivity_piezo_matrix(x0, y0, delta): # Sensitivity Matrix for only the piezo actuators
    n_inputs = 4  # Number of piezo actuators
    n_outputs = len(y0)

    M = np.zeros((n_outputs, n_inputs))  # Sensitivity Matrix Initialization

    piezo_serials = ops.piezo_serials_list  # Serial Numbers

    for j in range(n_inputs):
        x0_perturb = x0.copy()

        # Perturb piezo actuator j by delta
        ops.piezos.set_voltage(piezo_serials[j], x0_perturb[j] + delta)
        time.sleep(0.1)

        y_perturb = ops.quads.get_xy_position_tavg()  # Read qc values after perturbed voltage change [x1, y1, x2, y2]
        time.sleep(0.1)

        # Reset actuator j to initial value
        ops.piezos.set_voltage(piezo_serials[j], x0_perturb[j])
        time.sleep(0.1)

        # Compute and store the finite difference
        M[:, j] = (np.array(y_perturb) - np.array(y0)) / delta

    return M

In [ ]:
x0 = ops.piezos.get_all_voltages()
print(x0)
y0 = ops.quads.get_xy_position_tavg()
print(y0)

[50.1205481124302, 0.0457777642139958, 50.0030518509476, 0.0396740623187963]
[-0.05977722708822905, -0.03192378673665579, -0.011234367503891118, -0.05413412945951717]


In [ ]:
M = compute_sensitivity_piezo_matrix(x0, y0, delta)

29253216 set to 50.190740684225
29253216 set to 51.190740684225
29253216 set to 52.190740684225
29253216 set to 53.190740684225
29253216 set to 54.190740684225
29253216 set to 55.190740684225
29253216 set to 56.190740684225
29253216 set to 57.190740684225
29253216 set to 58.190740684225
29253216 set to 59.190740684225
29253216 set to 60.1205481124302
29253216 set to 60.2191228980377
29253216 set to 59.2191228980377
29253216 set to 58.2191228980377
29253216 set to 57.2191228980377
29253216 set to 56.2191228980377
29253216 set to 55.2191228980377
29253216 set to 54.2191228980377
29253216 set to 53.2191228980377
29253216 set to 52.2191228980377
29253216 set to 51.2191228980377
29253216 set to 50.2191228980377
29253216 set to 50.1205481124302
29253237 set to 0.0915555284279916
29253237 set to 1.0915555284279916
29253237 set to 2.0915555284279916
29253237 set to 3.0915555284279916
29253237 set to 4.091555528427992
29253237 set to 5.091555528427992
29253237 set to 6.091555528427992
29253237 

In [ ]:
print(M)
q = ops.quads.get_xy_position_tavg()
q = np.array(q)
print(q)

[[-1.14624940e-03  1.23075045e-05  1.05396350e-03  1.80731834e-05]
 [ 9.55521714e-05 -1.58636384e-03 -9.17423627e-05 -1.42769787e-03]
 [-2.84349583e-03 -2.42109561e-04  1.90373888e-03 -2.22894803e-04]
 [-5.65504318e-05 -2.51417017e-03 -3.32001648e-04 -2.34256105e-03]]
[-0.05937821 -0.03273285 -0.01214376 -0.05601849]


In [ ]:
M_subset = M[[0, 2], :][:, [0, 2]]  # shape: (2, 2) Only focusing on x-axis actuators and quadcells
q_subset = -q[[0, 2]]  # Result is shape (2, 1) Only the x-axis quadcells
print(M_subset, q_subset)

[[-0.00114625  0.00105396]
 [-0.0028435   0.00190374]] [0.05937821 0.01214376]


In [ ]:
solution = np.linalg.solve(M_subset, q_subset)
print(solution)

[123.02875302 190.13926307]


In [ ]:
current_volt = np.array(ops.piezos.get_all_voltages())[[0,2]]
print(current_volt)

for i in range(len(x_piezo_list)):
    ops.piezos.set_voltage(x_piezo_list[i], current_volt[i]+solution[i])

[50.12970367 50.00305185]
29253216 set to 50.129703665273
29253216 set to 49.129703665273
29253216 set to 48.129703665273
29253216 set to 47.129703665273
29253216 set to 46.129703665273
29253216 set to 45.129703665273
29253216 set to 44.129703665273
29253216 set to 43.129703665273
29253216 set to 42.129703665273
29253216 set to 41.129703665273
29253216 set to 40.129703665273
29253216 set to 39.129703665273
29253216 set to 38.129703665273
29253216 set to 37.129703665273
29253216 set to 36.129703665273
29253216 set to 35.129703665273
29253216 set to 34.129703665273
29253216 set to 33.129703665273
29253216 set to 32.129703665273
29253216 set to 31.129703665273
29253216 set to 30.129703665273
29253216 set to 29.129703665273
29253216 set to 28.129703665273
29253216 set to 27.129703665273
29253216 set to 26.129703665273
29253216 set to 25.129703665273
29253216 set to 24.129703665273
29253216 set to 23.129703665273
29253216 set to 22.129703665273
29253216 set to 21.129703665273
29253216 set t

Centering Quadcells Code

In [14]:
def shift_slope(current_volt_chan2, current_volt_chan4, threshold=0.1, final_limit=0.02,
                min_moves_chan2=4, min_moves_chan4=2, sig_strength = 0.08):
    step = 1
    stage_ser = ops.stages.serial_numbers[0]
    direction = 1 if ops.quads.get_xy_position(sig_strength)[0] > ops.quads.get_xy_position(sig_strength)[2] else -1

    print("Starting shift_slope...")

    while True:
        # --- chan2 loop (FAST axis, needs more pushes before switching) ---
        move_count = 0
        while True:
            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            total_deviation = abs(qc1) + abs(qc2)

            if total_deviation <= final_limit:
                print(f"Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return
            
            current_volt_chan2 += direction * step
            ops.stages.move_absolute(stage_ser, 'chan2', current_volt_chan2)
            time.sleep(0.3)
            move_count += 1

            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            print(f"[chan2] Moved to {current_volt_chan2}, QC: {qc1:.3f}, {qc2:.3f}")

            if move_count >= min_moves_chan2 and (abs(qc1) > threshold or abs(qc2) > threshold):
                print(f"Threshold hit on chan2 after {move_count} moves: QC1={qc1:.3f}, QC2={qc2:.3f}")
                break

        # --- chan4 loop (SLOW axis, fewer moves needed) ---
        move_count = 0
        while True:
            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            total_deviation = abs(qc1) + abs(qc2)

            if total_deviation <= final_limit:
                print(f"Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return
            
            current_volt_chan4 += direction * step
            ops.stages.move_absolute(stage_ser, 'chan4', current_volt_chan4)
            print(f"[chan4] Moved to {current_volt_chan4}, QC: {qc1:.3f}, {qc2:.3f}")
            move_count += 1
            time.sleep(0.3)

            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            print(f"[chan2] Moved to {current_volt_chan2}, QC: {qc1:.3f}, {qc2:.3f}")

            if move_count >= min_moves_chan4 and (abs(qc1) > threshold or abs(qc2) > threshold):
                print(f"Threshold hit on chan4 after {move_count} moves: QC1={qc1:.3f}, QC2={qc2:.3f}")
                break


In [122]:
print(ops.stages.get_all_positions())
#print(ops.quads.get_xy_position())

[0.0, 575.0, 850.0, 99.0]


In [123]:
stage_ser = ops.stage_serials[0]
ops.stages.move_absolute(stage_ser, 'chan4', 100)

Moving to position 100
Move Complete


In [13]:
qc_positions = ops.quads.get_xy_position(sig_strength=0.05)
qc1pos, qc2pos = qc_positions[0], qc_positions[2]
print(qc1pos, qc2pos)

Signal strength for 69253622: 0.054779888609140154
Signal strength for 69253977: 0.1496910048065919
-0.019531846064638203 -0.064


In [109]:
current_volt_chan2, current_volt_chan4 = ops.stages.get_all_positions()[1], ops.stages.get_all_positions()[3]
print(current_volt_chan2, current_volt_chan4)

552.0 99.0


In [111]:
shift_slope(current_volt_chan2, current_volt_chan4, sig_strength=0.045)

Signal strength for 69253622: 0.055848020141908906
Signal strength for 69253977: 0.1416037232013428
Signal strength for 69253622: 0.055848020141908906
Signal strength for 69253977: 0.1416037232013428
Starting shift_slope...
Signal strength for 69253622: 0.055848020141908906
Signal strength for 69253977: 0.1416037232013428
Signal strength for 69253622: 0.055848020141908906
Signal strength for 69253977: 0.1416037232013428
Moving to position 551
Move Complete
Signal strength for 69253622: 0.05691615167467765
Signal strength for 69253977: 0.14358739604791332
Signal strength for 69253622: 0.05691615167467765
Signal strength for 69253977: 0.14358739604791332
[chan2] Moved to 551.0, QC: -0.017, 0.114
Signal strength for 69253622: 0.05691615167467765
Signal strength for 69253977: 0.14358739604791332
Signal strength for 69253622: 0.05691615167467765
Signal strength for 69253977: 0.14358739604791332
Moving to position 550
Move Complete
Signal strength for 69253622: 0.059205004959182114
Signal st

ValueError: Signal too low for 69253977: 0.020904859998474096

In [124]:
ops.piezos.shutdown()
ops.quads.shutdown()
ops.stages.shutdown()

[INFO] 29253216 already near 0 (value = 0.027).
[OK] Shutdown piezo with serial 29253216
[INFO] 29253237 already near 0 (value = 0.027).
[OK] Shutdown piezo with serial 29253237
[INFO] 113250437 already near 0 (value = 0.000).
[OK] Shutdown piezo with serial 113250437
[INFO] 29253246 already near 0 (value = 0.031).
[OK] Shutdown piezo with serial 29253246
Shutdown aligner with serial 69253622
Shutdown aligner with serial 69253977
Shutdown linear stage with serial 97103046
